In [65]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

import numpy as np
import pandas as pd
import csv
import math
from collections import Counter
import matplotlib.pyplot as plt

df=pd.read_csv('D:/Script/knc/KoreanNewsChronicle/Topic_Abstract/Data/201912.csv',encoding='utf-8')

df = df.dropna()

# 불용어 제거
stopword = ['A씨', '만', '억', '조', '차', '그', '시', '대', '위', '분', '측', '곳']

for index, row in df.iterrows():
    topic = row['topics'].split(', ')
    for str in topic:
        if(str in stopword):
            topic.remove(str)

    topic_str = ", ".join(topic)
    df.loc[index, 'topics'] = topic_str

print('원래 기사 수 : ',len(df))
# 결측지 행 제거 후 topics에 있는 것을 추출
dp = df.duplicated(['headline'], keep='first')
count = 0
for b in dp:            #중복된 기사 제거
    if(b == True):
        df.drop(df.index[count], inplace = True)
        count -= 1
    count += 1

print('중복된 기사 제거후 기사수 : ',len(df))

tmp_index=[]

for index, row in df.iterrows():
    tmp = row['topics'].split(',')
    if len(tmp) < 4:
        tmp_index.append(index)

s = df.drop(tmp_index)

원래 기사 수 :  26515
중복된 기사 제거후 기사수 :  24497


In [66]:

news_topic = s['topics'].tolist()

# # CountVectrizer로 토큰화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(news_topic)

X = normalize(X)

neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X)
distances, indices = nbrs.kneighbors(X)


m = DBSCAN(eps=0.6, min_samples = 5)

m.fit(X)

labels = m.labels_

In [67]:
s['labels'] = '01_',labels

hot_topic_index = Counter(s['labels']).most_common()
hot_topic = dict(hot_topic_index)

print(hot_topic)

{-1: 22748, 6: 81, 13: 38, 7: 33, 37: 29, 26: 24, 20: 21, 31: 14, 10: 11, 5: 10, 22: 10, 33: 9, 8: 8, 15: 8, 50: 8, 30: 8, 43: 8, 3: 7, 18: 7, 32: 7, 40: 7, 44: 7, 0: 6, 1: 6, 2: 6, 4: 6, 9: 6, 21: 6, 19: 6, 24: 6, 29: 6, 34: 6, 46: 6, 48: 6, 14: 5, 23: 5, 11: 5, 12: 5, 16: 5, 17: 5, 25: 5, 27: 5, 28: 5, 39: 5, 35: 5, 36: 5, 38: 5, 41: 5, 42: 5, 45: 5, 47: 5, 49: 5}


In [68]:
count = []
for index, row in s['labels'].iteritems():
    if row == -1:
        s = s.drop(index)
    else:
        count.append(hot_topic[row])

s['cnt'] = count
s.sort_values(by='labels', ascending=False).to_csv('./Data/201912_cluster_DBSCAN2.csv', index=False, header=True,encoding="utf-8-sig")